<a href="https://colab.research.google.com/github/rekuantsun/Hoc-May/blob/main/k_fold_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Libraries


In [8]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

### Huấn luyện mô hình HQTT với 10-fold CV sử dụng Gradient Descent

In [5]:
def readData(filePath: str, filename: str):
  #Đọc dữ liệu
  D = np.loadtxt(os.path.join(filePath, filename), delimiter = ',')
  #Điều chỉnh dữ liệu - do đây là mô hình HQTT nên chấp nhận scale cả vector y
  scaler = MinMaxScaler()
  scaler.fit(D)
  D = scaler.transform(D)
  X, y = D[:, :-1], D[:, -1]
  x0 = np.ones((X.shape[0], 1))
  X = np.column_stack([x0, X])
  return X, y

def computeLoss(X, y, w):
  m = y.shape[0]
  J = 0
  h = np.dot(X, w)
  J = (1/(2*m))*np.sum(np.square(h - y))
  return J

def gradientDescent(X, y, w, alpha, n):
  m = y.shape[0]
  J_history = []
  w_optimal = w.copy()
  for i in range(n):
    h = np.dot(X, w_optimal)
    error = h - y
    w_optimal = w_optimal - (alpha/m)*np.dot(X.T, error)
    J_history.append(computeLoss(X=X, y=y, w=w_optimal))
  return w_optimal, J_history

def k_fold_10_GD(kf, n, alpha, step, w_opt, X_train, y_train, X):
  for train2_index, val_index in kf.split(X = X_train,y = y_train):
    step = step + 1
    print('\tBước lặp huấn luyện thứ: ', step)
    X_train2, X_val = X_train[train2_index], X_train[val_index]
    y_train2, y_val = y_train[train2_index], y_train[val_index]
    #Do chương trình chúng ta viết quy định y la mx1 nên cần reshape
    w_opt, J_history = gradientDescent(X=X_train2,y=y_train2.reshape((y_train2.shape[0],1)),
    w=w_opt, alpha=alpha, n=n)
    print('\t\tĐánh giá mô hình trên tập dữ liệu validation')
    y_hat = np.dot(X_val, w_opt)
    print('\t\t\tMSE: ', mean_squared_error(y_val, y_hat))

def main():
  #Bước 1: Đọc dữ liệu
  X, y = readData('/content', 'ex1data2.txt')
  #Bước 2: Phân chia train - test theo tỉ lệ 70% - 30%
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,
  random_state=15)
  #Bước 3: Xác định k-fold
  kf = KFold(n_splits=10)
  #Bước 4: Huấn luyện mô hình
  print('Huấn luyện mô hình với k-fold')
  w_opt = np.zeros((X.shape[1], 1))
  n = 1500
  alpha = 0.01
  step = 0
  k_fold_10_GD(kf, n, alpha, step, w_opt, X_train, y_train, X)
  #Bước 5: Kiểm định mô hình với tập dữ liệu test
  print('ĐÁNH GIÁ HIỆU NĂNG CỦA MÔ HÌNH TRÊN TẬP DỮ LIỆU TEST')
  y_hat = np.dot(X_test, w_opt)
  print('\tMSE: ', mean_squared_error(y_test, y_hat))
if __name__ == '__main__':
  main()

Huấn luyện mô hình với k-fold
	Bước lặp huấn luyện thứ:  1
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.007603148302747178
	Bước lặp huấn luyện thứ:  2
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.01042640957668497
	Bước lặp huấn luyện thứ:  3
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.005618257783946246
	Bước lặp huấn luyện thứ:  4
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.03219241151290909
	Bước lặp huấn luyện thứ:  5
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.010787307267518247
	Bước lặp huấn luyện thứ:  6
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.027773829789309024
	Bước lặp huấn luyện thứ:  7
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.023362977106992946
	Bước lặp huấn luyện thứ:  8
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.01003361495747874
	Bước lặp huấn luyện thứ:  9
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.021680569897733263
	Bước lặp huấn luyệ

### Huấn luyện mô hình HQTT với 10-fold CV sử dụng LinearRegression 

In [6]:
def readData(filePath: str,filename: str):
  D = np.loadtxt(os.path.join('/content', 'ex1data2.txt'), delimiter=',')
  scaler = MinMaxScaler()
  scaler.fit(D)
  D = scaler.transform(D)
  X, y = D[:, :-1], D[:, -1]
  return X, y

def k_fold_10_linear(kf, X_train, y_train, X, model, step):
  for train2_index, val_index in kf.split(X = X_train,y = y_train):
    step = step + 1
    print('\tBước lặp huấn luyện thứ: ', step)
    w_opt = np.zeros((X.shape[1], 1))
    X_train2, X_val = X_train[train2_index], X_train[val_index]
    y_train2, y_val = y_train[train2_index], y_train[val_index]
    model.fit(X_train2, y_train2)
    print('\t\tĐánh giá mô hình trên tập dữ liệu validation')
    y_hat = model.predict(X_val)
    print('\t\t\tMSE: ', mean_squared_error(y_val, y_hat))

def main():
  #Bước 1: Đọc dữ liệu
  X, y = readData('/content','ex1data2.txt')
  #Bước 2: Phân chia train - test theo tỉ lệ 70% - 30%
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=15)
  #Bước 3: Xác định k-fold
  kf = KFold(n_splits=10)
  #Bước 4: Huấn luyện mô hình
  print('Huấn luyện mô hình LinearRegression với k-fold')
  model = LinearRegression()
  step = 0
  k_fold_10_linear(kf, X_train, y_train, X, model, step)
  #Bước 5: Kiểm định mô hình với tập dữ liệu test
  print('ĐÁNH GIÁ HIỆU NĂNG CỦA MÔ HÌNH TRÊN TẬP DỮ LIỆU TEST')
  y_hat = model.predict(X_test)
  print('\tMSE: ', mean_squared_error(y_test, y_hat))
if __name__ == "__main__":
  main()

Huấn luyện mô hình LinearRegression với k-fold
	Bước lặp huấn luyện thứ:  1
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.017411617285763195
	Bước lặp huấn luyện thứ:  2
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.007521757373056872
	Bước lặp huấn luyện thứ:  3
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.003876138559617204
	Bước lặp huấn luyện thứ:  4
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.021833013405928244
	Bước lặp huấn luyện thứ:  5
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.010291952905878145
	Bước lặp huấn luyện thứ:  6
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.021192949345013895
	Bước lặp huấn luyện thứ:  7
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.04349119371377332
	Bước lặp huấn luyện thứ:  8
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.015677122297274936
	Bước lặp huấn luyện thứ:  9
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.019790401558297
	Bư

### Huấn luyện mô hình HQTT với 10-fold CV sử dụng Lasso


In [7]:
def readData(filePath: str,filename: str):
  #Bước 1: Đọc dữ liệu
  D = np.loadtxt(os.path.join('/content', 'ex1data2.txt'), delimiter=',')
  #Bước 2: Điều chỉnh dữ liệu - do đây là mô hình HQTT nên chấp nhận scale cả vector y
  scaler = MinMaxScaler()
  scaler.fit(D)
  D = scaler.transform(D)
  #Bước 3: Phân chia train - test theo tỉ lệ 70% - 30%
  X, y = D[:, :-1], D[:, -1]
  return X, y

def k_fold_10_lasso(kf, X_train, y_train, X, model, step, alpha):
  for train2_index, val_index in kf.split(X = X_train,y = y_train):
    step = step + 1
    print('\tBước lặp huấn luyện thứ: ', step)
    w_opt = np.zeros((X.shape[1], 1))
    X_train2, X_val = X_train[train2_index], X_train[val_index]
    y_train2, y_val = y_train[train2_index], y_train[val_index]
    model.fit(X_train2, y_train2)
    print('\t\tĐánh giá mô hình trên tập dữ liệu validation')
    y_hat = model.predict(X_val)
    print('\t\t\tMSE: ', mean_squared_error(y_val, y_hat))

def main():
  #Bước 1: Đọc dữ liệu
  X, y = readData("/content","ex1data2.txt")
  #Bước 2: Phân chia train - test theo tỉ lệ 70% - 30%
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=15)
  #Bước 3: Xác định k-fold
  kf = KFold(n_splits=10)
  #Bước 4: Huấn luyện mô hình
  print('Huấn luyện mô hình Lasso với k-fold')
  model = Lasso()
  step = 0
  alpha = 0.1
  k_fold_10_lasso(kf, X_train, y_train, X, model, step, alpha)
  #Bước 5: Kiểm định mô hình với tập dữ liệu test
  print('ĐÁNH GIÁ HIỆU NĂNG CỦA MÔ HÌNH TRÊN TẬP DỮ LIỆU TEST')
  y_hat = model.predict(X_test)
  print('\tMSE: ', mean_squared_error(y_test, y_hat))

if __name__ == "__main__":
  main()

Huấn luyện mô hình Lasso với k-fold
	Bước lặp huấn luyện thứ:  1
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.004447991101057281
	Bước lặp huấn luyện thứ:  2
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.036569107012577116
	Bước lặp huấn luyện thứ:  3
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.1732979608817463
	Bước lặp huấn luyện thứ:  4
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.11437671688754669
	Bước lặp huấn luyện thứ:  5
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.08016355660648979
	Bước lặp huấn luyện thứ:  6
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.020932247777911358
	Bước lặp huấn luyện thứ:  7
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.06503157574039026
	Bước lặp huấn luyện thứ:  8
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.03322813964126265
	Bước lặp huấn luyện thứ:  9
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.0595635756156567
	Bước lặp huấn luy

### Huấn luyện mô hình HQTT với 10-fold CV sử dụng Ridge


In [9]:
def readData(filePath: str,filename: str):
  #Bước 1: Đọc dữ liệu
  D = np.loadtxt(os.path.join('/content', 'ex1data2.txt'), delimiter=',')
  #Bước 2: Điều chỉnh dữ liệu - do đây là mô hình HQTT nên chấp nhận scale cả vector y
  scaler = MinMaxScaler()
  scaler.fit(D)
  D = scaler.transform(D)
  #Bước 3: Phân chia train - test theo tỉ lệ 70% - 30%
  X, y = D[:, :-1], D[:, -1]
  return X, y

def k_fold_10_ridge(kf, X_train, y_train, X, model, step, alpha):
  for train2_index, val_index in kf.split(X = X_train,y = y_train):
    step = step + 1
    print('\tBước lặp huấn luyện thứ: ', step)
    w_opt = np.zeros((X.shape[1], 1))
    X_train2, X_val = X_train[train2_index], X_train[val_index]
    y_train2, y_val = y_train[train2_index], y_train[val_index]
    model.fit(X_train2, y_train2)
    print('\t\tĐánh giá mô hình trên tập dữ liệu validation')
    y_hat = model.predict(X_val)
    print('\t\t\tMSE: ', mean_squared_error(y_val, y_hat))

def main():
  #Bước 1: Đọc dữ liệu
  X, y = readData("/content","ex1data2.txt")
  #Bước 2: Phân chia train - test theo tỉ lệ 70% - 30%
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=15)
  #Bước 3: Xác định k-fold
  kf = KFold(n_splits=10)
  #Bước 4: Huấn luyện mô hình
  print('Huấn luyện mô hình Ridge với k-fold')
  model = Ridge()
  step = 0
  alpha = 1
  k_fold_10_lasso(kf, X_train, y_train, X, model, step, alpha)
  #Bước 5: Kiểm định mô hình với tập dữ liệu test
  print('ĐÁNH GIÁ HIỆU NĂNG CỦA MÔ HÌNH TRÊN TẬP DỮ LIỆU TEST')
  y_hat = model.predict(X_test)
  print('\tMSE: ', mean_squared_error(y_test, y_hat))

if __name__ == "__main__":
  main()

Huấn luyện mô hình Ridge với k-fold
	Bước lặp huấn luyện thứ:  1
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.007342095872437096
	Bước lặp huấn luyện thứ:  2
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.014049333078351068
	Bước lặp huấn luyện thứ:  3
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.04428852697107722
	Bước lặp huấn luyện thứ:  4
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.05069037283758421
	Bước lặp huấn luyện thứ:  5
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.028517646668885285
	Bước lặp huấn luyện thứ:  6
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.03465955433033329
	Bước lặp huấn luyện thứ:  7
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.022081137144973808
	Bước lặp huấn luyện thứ:  8
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.014284963922507813
	Bước lặp huấn luyện thứ:  9
		Đánh giá mô hình trên tập dữ liệu validation
			MSE:  0.0437355963026947
	Bước lặp huấn 